In [1]:
%matplotlib inline
from IPython.display import Image
import numpy.matlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy.stats
import openpyxl
from scipy.optimize import curve_fit
from mpl_toolkits.mplot3d import Axes3D

import xml.etree.ElementTree as ET
from subprocess import call
import vtk
from vtk.util.numpy_support import vtk_to_numpy
from scipy.interpolate import griddata

import openmc

import Define_Nektar
import Define_OpenMC
import CouplingDepletion

import os
import time
import shutil

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Parameters of reactor
# Unit: cm
# To be discussed: Parameters for reactor to become critical
parameters_dic = {}

parameters_dic.update(fuel_r = 11/2)
parameters_dic.update(fuel_h = 19.5)

parameters_dic.update(controlRod_r = 4.4/2)
parameters_dic.update(controlRod_h_max = 27)
parameters_dic.update(controlRod_l = 24.5)

parameters_dic.update(reflector_r = 42/2)

parameters_dic.update(heat_pipe_R = 1.27/2)
parameters_dic.update(heat_pipe_r = 1.27/2-0.1)

parameters_dic.update(top_distance = 10.5)
parameters_dic.update(bottom_distance = 5)

parameters_dic.update(heat_power = 4000)

parameters_dic.update(reflector_h = parameters_dic['top_distance']+parameters_dic['bottom_distance']+parameters_dic['fuel_h'])

temp_pipe = 1073.5
# Insert control rod, maximum is 27
controlRod_deep = 0
empty_reflector_height = 0
# Number of cells
cells_num_dic = {'n_r':1,'n_r_outer':1,'n_h':1}
# Settings of OpenMC
settings_MC_dic = {'batches':150,'inactive':50,'particles':1000,'temperature_update':True}
#Settings of Nektar++
settings_nek_dic = {'file_name':'HeatPipeReactor_3','solver_name':'ADRSolver'}
# fiel_name.xml with settings of Poisson solver
# Settings of Coupling
settings_dic = {'settings_MC_dic':settings_MC_dic,'settings_nek_dic': settings_nek_dic}
settings_dic.update(temperature_update = False)
settings_dic.update(iteration = 3)
settings_dic.update(time_steps = [30 * 24 * 60 * 60] * 3)
settings_dic.update(chain = 'chain_casl_sfr.xml')
# Initial temperature distribution in cells
Initial_temp = 1073.5
# Get some data
# volume_mat,fuel_cell_ID_list = Define_OpenMC.define_Geo_Mat_Set(cells_num_dic,parameters_dic,settings_dic,temp_cells_mat,controlRod_deep,empty_reflector_height)

In [ ]:
CouplingDepletion.main_Depletion(cells_num_dic,parameters_dic,settings_dic,Initial_temp,controlRod_deep,empty_reflector_height)

====================Burn-up step1 begins====================
Asuka
====================Iteration1 begins====================
Saito
Momoko
Iteration 1: OpenMC run time:203.0220170021057
Iteration 1: k_eff:1.0052909747481868 k_eff_dev:0.002752657393160392
Iteration 1: OpenMC post-process time:31.294835567474365
Iteration 1 Nektar run time:572.1888036727905
Iteration 1 Nektar post-process time:2.5610575675964355
Maximum relative error: 0.01998498320407417
Total time: 810.0182843208313
Momoko
====================Iteration2 begins====================
Asuka
Momoko
Iteration 2: OpenMC run time:157.67428731918335
Iteration 2: k_eff:1.003951970480911 k_eff_dev:0.002741643803066174
Iteration 2: OpenMC post-process time:19.16286540031433
Iteration 2 Nektar run time:216.99044227600098
Iteration 2 Nektar post-process time:2.050424098968506
Maximum relative error: 0.0003734300596121091
Total time: 396.83219170570374
====================Burn-up step2 begins====================
Hinako
================

In [4]:
results = openmc.deplete.ResultsList.from_hdf5("./depletion_results.h5")

In [5]:
time, k = results.get_eigenvalue()
k

array([[1.00749217e+00, 9.38282831e-04],
       [1.00749976e+00, 1.29555455e-03],
       [1.00725055e+00, 1.19419465e-03],
       [1.00685345e+00, 1.30932293e-03],
       [1.00772421e+00, 1.25013080e-03]])

In [3]:
x,y,z = Define_Nektar.readNodesFromVtu(file_name)
nodes_dic = {'x':x,'y':y,'z':z}
# Initial temperature distribution in nodes
temp_nodes_vec = Initial_temperature*np.ones(len(x))
fuel_nodes_index = Define_OpenMC.getFuelNodesIndex(nodes_dic,parameters_dic)
# For Test
temp_error_vec = np.zeros(iteration)
temp_ave_error_vec = np.zeros(iteration)

k_eff_mean_vec = np.zeros(iteration)
k_eff_dev_vec = np.zeros(iteration)

heat_ratio = np.zeros(iteration)
flux_ratio = np.zeros(iteration)

num_col = cells_num_dic['n_h']*(cells_num_dic['n_r']+cells_num_dic['n_r_outer'])

heat_mean_mat = np.zeros((iteration,num_col))
heat_dev_mat = np.zeros((iteration,num_col))
flux_mean_mat = np.zeros((iteration,num_col))
flux_dev_mat = np.zeros((iteration,num_col))

In [ ]:
for i in range(iteration):
    start_tot = time.time()
    # For Test
    temp_cells_mat_last = temp_cells_mat
    
    # Iteration
    print('Iteration: '+str(i+1)+' begins')
    # Run OpenMC !
    time_start = time.time()
    openmc.run(output=False)
    time_end = time.time()
    print('Iteration: '+str(i+1)+' OpenMC run time:'+str(time_end-time_start))
    
    # Post-process the result of heat source and generate Force.pts
    time_start = time.time()
    k_eff,tally_dic = Define_OpenMC.postProcess(nodes_dic,volume_mat,temp_nodes_vec,fuel_nodes_index,parameters_dic,cells_num_dic,settings_dic,fuel_cell_ID_list)
    time_end = time.time()
    print('Iteration: '+str(i+1)+' OpenMC post-process time:'+str(time_end-time_start))
    
    # k-eff: mean value and standard deviation
    k_eff_mean_vec[i] = k_eff.nominal_value
    k_eff_dev_vec[i] = k_eff.std_dev
    print('k_eff_mean: '+str(k_eff_mean_vec[i]))
    print('k_eff_dev: '+str(k_eff_dev_vec[i]))
  
    # Heat and flux    
    heat_mean_mat[i,:] = tally_dic['heat_mean']
    heat_dev_mat[i,:] = tally_dic['heat_dev']
    flux_mean_mat[i,:] = tally_dic['flux_mean']
    flux_dev_mat[i,:] = tally_dic['flux_dev']
    
    if i>0:
        heat_index = np.where((heat_mean_mat[i,:]<heat_mean_mat[i-1,:]+heat_dev_mat[i-1,:]) & (heat_mean_mat[i,:]>heat_mean_mat[i-1,:]-heat_dev_mat[i-1,:]))
        flux_index = np.where((flux_mean_mat[i,:]<flux_mean_mat[i-1,:]+flux_dev_mat[i-1,:]) & (flux_mean_mat[i,:]>flux_mean_mat[i-1,:]-flux_dev_mat[i-1,:]))
        heat_ratio[i] = len(heat_index[0])/num_col
        flux_ratio[i] = len(flux_index[0])/num_col
    
    print('heat_ratio: '+str(heat_ratio[i]))
    print('flux_ratio: '+str(flux_ratio[i]))
    
    # Run Nektar++
    if os.path.exists(file_name+'.fld'):
        os.remove(file_name+'.fld')
    if os.path.exists(file_name+'.vtu'):
        os.remove(file_name+'.vtu')    
    
    time_start = time.time()
    file_name_new = Define_Nektar.runNektar_Temp(file_name,solver_name,temp_pipe,i)
    time_end = time.time()
    print('Iteration: '+str(i+1)+' Nektar run time:'+str(time_end-time_start))
        
    # Post-process result of temperature
    time_start = time.time()
    temp_nodes_vec = Define_Nektar.postProcess_Temp(file_name_new)
    time_end = time.time()
    print('Iteration: '+str(i+1)+' Nektar post-process time:'+str(time_end-time_start))
    temp_cells_mat = Define_OpenMC.getCellTemperature(nodes_dic,temp_nodes_vec,fuel_nodes_index,parameters_dic,cells_num_dic)

    volume_mat,fuel_cell_ID_list = Define_OpenMC.define_Geo_Mat_Set(cells_num_dic,parameters_dic,settings_dic,temp_cells_mat,controlRod_deep)
    # For Test
    
    # Largest relative error of fuel temperature
    temp_error = np.abs(temp_cells_mat-temp_cells_mat_last)/temp_cells_mat_last
    temp_error_vec[i] = temp_error.max()
    print('Maximum relative error: '+str(temp_error_vec[i]))
    
    # Average relative error of fuel temperature
    temp_ave_error_vec[i] = temp_error.sum()/(np.size(temp_error,0)*np.size(temp_error,1))
    print('Average relative error: '+str(temp_ave_error_vec[i]))
    
    end_tot = time.time()
    print('Total time: '+str(end_tot-start_tot))
    print('======================================')

In [1]:
import datetime

In [2]:
now_time = datetime.datetime.now()

In [3]:
now_time

datetime.datetime(2020, 2, 20, 12, 58, 18, 626906)

In [5]:
time1_str = datetime.datetime.strftime(now_time,'%Y_%m_%d_%H_%M_%S')

In [6]:
time1_str

'2020_02_20_12_58_18'

In [7]:
import os

In [12]:
os.mkdir(time1_str)

In [9]:
b = "burn-up"

In [10]:
file_path = b + time1_str

In [11]:
print(file_path)

burn-up2020_02_20_12_58_18


In [13]:
import shutil

In [15]:
shutil.copy('chain_casl_sfr.xml',time1_str)

'2020_02_20_12_58_18/chain_casl_sfr.xml'